In [1]:
# --- Local Setup ---
import os
import sys
from pathlib import Path

# Add repository root to path
# Assuming notebook is in notebooks/ and repo root is one level up
repo_root = Path("..").resolve()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

# Change working directory to repo root so config files are found correctly
os.chdir(repo_root)

In [3]:
# setup Environment & Config
import yaml
from src.config import load_settings

settings = load_settings()
print(f"Data Root: {settings.paths.data}")
print(f"Manga Dir: {settings.paths.manga_dir}")
print(f"Annotations Dir: {settings.paths.annotations_dir}")
print(f"Models Dir: {settings.paths.model_dir}")
print(f"Runs Dir: {settings.paths.runs_dir}")
print(f"Output Dir: {settings.paths.output_dir}")

Data Root: catnip-data
Manga Dir: catnip-data/data/manga
Annotations Dir: catnip-data/data/annotations
Models Dir: catnip-data/models
Runs Dir: catnip-data/runs
Output Dir: catnip-data/results


In [ ]:
# run inference on the entire dataset
from ultralytics import YOLO
from src.output.output import save_inference_results

# load the trained model
model_path = settings.paths.model_dir / "izutsumi_v2_local_251227" / "weights" / "best.pt"
model = YOLO(model_path)

# define output directory mapping to ./catnip-data/results/
results_dir = settings.paths.output_dir
results_dir.mkdir(parents=True, exist_ok=True)

# Use the actual manga directory directly
inference_source = settings.paths.manga_dir

# Create a specific output directory for flattened results
output_dir = results_dir / "inference"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"running inference on {inference_source}...")

# run prediction recursively
results = model.predict(
    source=str(inference_source) + "/**/*.*",  # recursive glob pattern
    project=str(results_dir),
    name="inference",
    save=False,      # Disable auto-save to handle manually
    save_txt=False,  # Disable auto-txt to handle manually
    conf=0.25,      # confidence threshold
    stream=True     # use generator to handle large datasets
)

save_inference_results(results, output_dir, inference_source)